# TESTING AND DEVELOPEMENT NOTEBOOK

_Paul-Emmanuel SOTIR_  

In [ ]:
# Imports
import re
import os
import sys
import inspect
import logging
import threading
import subprocess
from pathlib import Path
from typing import Union, Dict, Any, Tuple, Optional

In [ ]:

def start_tensorboard_server(logdir: Union[Path, str], port: Union[int, str], print_server_url: bool = True) -> Tuple[str, bool]:
    """ Non-Blocking call to tensorboard in a child process.
    Returns popen subprocess object. Call popen object's `terminate` function in order to stop tensorboard server (prefered way over `deepcv.utils.stop_any_tensorboard_server`). 
    """
    sub = subprocess.Popen(['tensorboard', '--logdir', logdir, '--port', port], stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True, check=False)
    if print_server_url:
        logging.info(f'Started Tensorboard server with `logdir={logdir}`, browse to "http://localhost:{port}".')

popen.terminate()


def stop_any_tensorboard_server(port: Union[int, str], cmd_must_contain: Optional[str] = 'tensorboard') -> bool:
    success = False
    if sys.platform == 'linux' or sys.platform == 'linux2' or sys.platform == 'darwin':
        # Find any process listenning on given port by parsing lsof output
        sub = subprocess.run(['lsof', '+M', '-P', '-w', '-n', '-Fpc', '-i', f'TCP:{port}'], stdout=subprocess.PIPE, universal_newlines=True, check=True)
        pids = re.findall('p([0-9]+)\r?\n', sub.stdout, re.MULTILINE)
        comands = re.findall('c([0-9]+)\r?\n', sub.stdout, re.MULTILINE)

        # Stop all Tensorboard server(s) listenning on given port by killing its process(es)
        for cmd, server_pid in zip(comands, pids):
            if not cmd_must_contain or cmd_must_contain in cmd.lower():
                sub = subprocess.run(['kill', '-9', f'{server_pid}'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True, check=False)
                if sub.returncode == 0:
                    success = True
                    logging.info(f'Sucessfully terminated tensorboard server listenning on port "{port}" with process pid "{server_pid}".')
                else:
                    logging.warn(f'Warning: `stop_tensorboard_server` failed to stop a Tensorboard server listenning on port "{port}": '
                                 f'`kill -9 {server_pid}` returned non-zero status "{sub.returncode}", `stderr="{sub.stderr}"`.')
    else:
        logging.warn(f'Warning: `stop_tensorboard_server` function cant stop Tensorboard server on "{sys.platform}" OS platform (relies on `lsof` and `kill` Unix commands)')
    return success

In [ ]:
start_tensorboard_server(r'../../data/04_training/', port=5005)